In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import scale
import datetime
import importlib
import global_settings
importlib.reload(global_settings)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Define working folder

setts = global_settings.Settings()

root_folder = setts.root_folder
tmp1_folder = setts.tmp1_folder
tmp2_folder = setts.tmp2_folder
output_folder = setts.output_folder

# tmp1_folder = root_folder + "/data/tmp"
# tmp2_folder = root_folder + "/data/tmp2"
# input_folder = root_folder + "/input"
# output_folder = root_folder + "/output"

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)
    
year_limit = setts.year_limit

Directory found: D:\Software\temp\CeDX\data_2024_05/data
Directory found: D:\Software\temp\CeDX\data_2024_05/data/tmp
Directory found: D:\Software\temp\CeDX\data_2024_05/data/tmp2
Directory found: D:\Software\temp\CeDX\data_2024_05/output


In [3]:
print(setts.root_folder)
print(setts.month_to)
print(setts.month_last)

D:\Software\temp\CeDX\data_2024_05
20240501
20240430


In [4]:
import datetime
import time

df_full = pd.read_csv(output_folder + '/dataset_all_full_columns.csv')
current_date = setts.month_last_dt

print(current_date)

#df_full['am_age'] = pd.to_datetime(df_full['START_date'], format="%m/%d/%Y").dt.date
df_full['am_age'] = pd.to_datetime(df_full['START_date'], format="%Y-%m-%d").dt.date

df_full['am_age'] = df_full['am_age'].apply(lambda x: ((current_date.year - x.year) * 12 + (current_date.month - x.month)) / 12)



2024-04-30 00:00:00


In [5]:
use_cols = ['UID', 'DEALER_CODE', 'DOC_DATE', 'MODEL', 'MODEL_YEAR', 'WARRANTY_START_DATE','TTL','START_year']
#service = pd.read_csv(output_folder + '/merged_sale_service.csv', usecols = use_cols)

service = pd.read_csv(root_folder + '/data/service_data.csv')
service['DOC_DATE'] = pd.to_datetime(service['DOC_DATE'], format='%Y%m%d', errors='coerce')
service = service[(service['DOC_DATE'] <= setts.month_last_dt)]

service['DROP_OFF_DATE'] = pd.to_datetime(service['DROP_OFF_DATE'], format='%Y%m%d', errors='coerce')
service['PICK_UP_DATE'] = pd.to_datetime(service['PICK_UP_DATE'], format='%Y%m%d', errors='coerce')
service['DOC_year'] = service['DOC_DATE'].dt.year.fillna(0)
service['GROSS_VALUE'] = service['GROSS_VALUE'].apply(lambda x: 0 if x < 0 else float(x))
service = service[(service['DOC_year'] > 2015) & (service['DOC_year'] <= year_limit)]

sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')
sale['MODEL'] = sale['MODEL_TEXT_1'].str.split(' ').str[0]
sale = sale.drop('MODEL_TEXT_1', axis=1)
sale['WARRANTY_START_DATE'] = pd.to_datetime(sale['WARRANTY_START_DATE'], format='%Y-%m-%d')
sale['START_year'] = sale['WARRANTY_START_DATE'].dt.year
sale = sale[(sale['START_year'] > 2015) & (sale['START_year'] <= year_limit)]
# sale = sale.drop(['DEALER_CODE', 'customer_index', 'Cust_group'], axis=1)
# sale = sale.drop(['DEALER_CODE'], axis=1)

cb = pd.merge(sale, right=service, how='left', left_on= 'VIN',right_on='VIN')
service = cb

C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_24164\4269404092.py:4: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  service = pd.read_csv(root_folder + '/data/service_data.csv')
C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_24164\4269404092.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')


In [6]:
service['last_service_age'] = pd.to_datetime(service['DOC_DATE'], format="%Y-%m-%d").dt.date
service['last_service_age'] = service['last_service_age'].apply(lambda x: ((current_date.year - x.year) * 12 + (current_date.month - x.month)) / 12)
service['last_service_age'] = service['last_service_age'].round(1)

In [7]:
s = service[service['last_service_age'] == service.groupby('UID')['last_service_age'].transform('min')]
s = s.sort_values(by='WARRANTY_START_DATE')
s = s.drop_duplicates(subset=['UID'], keep='first')
s['last_DEALER_CODE'] = s['DEALER_CODE_y']
s['DEALER_CODE'] = s['DEALER_CODE_x']

print(s)

          CUSTOMER_CODE_x  TITLE            CUSTOMER_NAME OCCUPATION  \
171            2000073677    2.0  VŨ VĂN HỢP+ VŨ VĂN HỢP+        NaN   
618            2000063096    2.0            Phạm Văn Sinh        NaN   
731            2000053117    2.0          NGUYỄN VĂN TRẬN        NaN   
919            2000057546    2.0             PHẠM ĐỨC YÊN        NaN   
529            2000067131    2.0           ĐẶNG HỮU THANH        NaN   
...                   ...    ...                      ...        ...   
11697182       2000319104    2.0               Lê Văn Đức        NaN   
11697183       2000319105    2.0         Nguyễn Công Tuấn        NaN   
11697184       2000319106    2.0         Nguyễn Thanh Sơn        NaN   
11697177       2000319099    2.0            Ngô Văn Quyết        NaN   
11697204       2000319026    2.0           phan thuỳ linh        NaN   

                    DISTRICT  \
171                 Biên Hòa   
618            Buôn Ma Thuột   
731                   Đất Đỏ   
919    

In [8]:
df=pd.read_excel(output_folder + '/dataset_all_scored_info_RFM_result.xlsx')
df = pd.merge(df, right=df_full[['UID', 'visit_num','am_age','TTL','START_year']], how='left', left_on= 'UID',right_on='UID')
df = pd.merge(df, right=s[['UID', 'VIN','DEALER_CODE', 'last_DEALER_CODE' ,'last_service_age']], how='left', left_on= 'UID',right_on='UID')

df = df.drop_duplicates(subset='UID', keep='last')

In [9]:
print(df.columns)

Index(['UID', 'keika_max', 'total_buy', 'TIRE', 'BATTERY', 'visit_rate',
       'SA_cont', 'mp', 'MH_ID', 'chat', 'CR', 'total_satisfaction', 'ew_num',
       'coupon_use', 'SA_select', 'app_reserve_mean', 'estimate_cost',
       'view_product_detail', 'view_test_drive', 'notif_v_spa', 'notif_vpi',
       'notif_vpm', 'PM2ov', 'Score', 'CUSTOMER_NAME', 'GENDER', 'CITY',
       'Customer segmentation', 'visit_num', 'am_age', 'TTL', 'START_year',
       'VIN', 'DEALER_CODE', 'last_DEALER_CODE', 'last_service_age'],
      dtype='object')


In [10]:
df.drop(columns=['Customer type','PM2ov','CR','view_test_drive'],inplace=True, errors='ignore')
df.to_excel(output_folder + '/dataset_all_scored_info_RFM_result_aged.xlsx', index=False)
del service

In [11]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

norm_col = ['keika_max', 'TIRE', 'BATTERY', 'visit_rate',
       'SA_cont', 'mp', 'MH_ID', 'chat', 'total_satisfaction', 'ew_num',
       'coupon_use', 'SA_select', 'app_reserve_mean', 'view_product_detail',
       'notif_v_spa', 'notif_vpi', 'notif_vpm', 'estimate_cost']
df_norm = df[norm_col]

scaler = StandardScaler()
sc = scaler.fit_transform(df_norm)
df_norm = pd.DataFrame(sc, index=df_norm.index, columns=df_norm.columns)

df_norm=(df_norm-df_norm.min())/(df_norm.max()-df_norm.min())


In [12]:
df_c = df.copy()
df_c[norm_col] = df_norm[norm_col]
df_c['visit_rate_by_year']=df_c['visit_num']/df_c['am_age']
#pd.to_datetime(df_full['START_date'], format="%m/%d/%Y").dt.date.apply(lambda x: ((current_date.year - x.year) * 12 + (current_date.month - x.month)) / 12)

df_c['am_age'] = np.floor(df_c['am_age'])
df_c['am_age'] = df_c['am_age'].replace(0, 'Under 1 year')


df_c['Region'] = df_c['DEALER_CODE'].astype(str).apply(lambda x: 'North' if x[0] == '1' else 'Central' if x[0] == '2' else 'South')
df_c.insert(0, "Date", datetime.datetime.strptime(str(setts.month_last), "%Y%m%d"))
df_c = df_c.rename(columns={'DEALER_CODE' : 'Dealer', 'TTL': 'Revenue', 'START_year' : 'From','Customer segmentation' : 'HM Segment'})
df_c.columns
df_c.to_excel(output_folder + '/dataset_all_scored_info_RFM_result_aged_normalized.xlsx', index=False)
df_c


,Date,UID,keika_max,total_buy,TIRE,BATTERY,visit_rate,SA_cont,mp,MH_ID,...,visit_num,am_age,Revenue,From,VIN,Dealer,last_DEALER_CODE,last_service_age,visit_rate_by_year,Region
0,2024-04-30,2,1.000,1,0.0,0.000000,0.500000,0.0,1.0,0.0,...,9,7.0,39767635.0,2016,RLHRM1838GY701123,1040,1040.0,3.8,1.200000,North
1,2024-04-30,4,1.000,1,0.0,0.090909,0.500000,0.0,1.0,0.0,...,16,7.0,59427443.0,2016,RLHGM6665GY701758,1040,1040.0,3.5,2.133333,North
2,2024-04-30,5,1.000,1,0.0,0.181818,0.750000,0.0,1.0,0.0,...,18,7.0,69957978.0,2016,RLHRM183XGY701110,1040,1040.0,0.2,2.400000,North
4,2024-04-30,12,0.375,2,0.0,0.000000,0.666667,1.0,0.0,0.0,...,15,2.0,15787052.0,2021,RLHGN2661MY001558,3030,3030.0,1.3,5.806452,South
5,2024-04-30,262,0.750,2,0.0,0.000000,0.833333,0.0,0.0,0.0,...,16,6.0,18992094.0,2018,RLHGM6666JY801679,1090,1090.0,1.0,2.560000,North
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154265,2024-04-30,935907235098,1.000,1,0.0,0.181818,0.625000,0.0,0.0,0.0,...,12,7.0,27105038.0,2016,RLHRM1837GY700738,2010,2060.0,0.2,1.582418,Central
154266,2024-04-30,947624415097,0.625,1,0.0,0.090909,0.800000,0.0,0.0,0.0,...,12,5.0,24842751.0,2019,MRHRU5850KP080077,2070,2070.0,0.8,2.360656,Central
154267,2024-04-30,985740638038,0.750,1,0.0,0.000000,0.833333,0.0,0.0,0.0,...,19,5.0,36144516.0,2018,MRHRW1870JP082156,1140,1140.0,0.0,3.257143,North
154268,2024-04-30,987578246098,1.000,1,0.0,0.090909,0.750000,0.0,0.0,0.0,...,22,7.0,45135880.0,2016,RLHGM6666GY603922,1060,1110.0,0.2,2.838710,North


In [13]:
df_c_rearrange = df_c.copy()
df_out_rearrange = pd.DataFrame()

output_cols = ['Date', 'UID', 'keika_max', 'total_buy', 'TIRE', 'BATTERY', 'visit_rate', 'SA_cont', 'mp', 'MH_ID', 'chat', 'NULL0', 'total_satisfaction', 'ew_num', 'coupon_use', 'SA_select', 'app_reserve_mean', 'estimate_cost', 'view_product_detail', 'NULL1', 'notif_v_spa', 'notif_vpi', 'notif_vpm', 'NULL2', 'Score', 'CUSTOMER_NAME', 'GENDER', 'CITY', 'NULL3', 'NULL4', 'NULL5', 'NULL6', 'NULL7', 'Revenue', 'HM Segment', 'Dealer', 'Region', 'From', 'visit_num', 'am_age', 'visit_rate_by_year', 'last_DEALER_CODE', 'last_service_age']

for col in output_cols:
    if col.startswith("NULL"):
        df_out_rearrange[col] = ''
    else:
        df_out_rearrange[col] = df_c_rearrange[col]

df_out_rearrange.to_excel(output_folder + '/dataset_all_scored_info_RFM_result_aged_normalized_rearranged.xlsx', index=False)
